**Part II**

In this part, we focus on convolutional layers, which are the basic building blocks of classical architectures such as VGG and ResNet.
To understand their effect, we manipulate them a bit and introduce the maxpooling operations often associated with them (**A**). Then, we observe how their parameters have converged after training on the ImageNet database (**B**). We also visualize the output signal of the convolutional layers (**C**).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

import torch
import torch.nn as nn   # pre-defined layers

from torch.utils.data import Dataset, DataLoader

# to load pre-trained models on ImageNet:
import torchvision
from torchvision import datasets, models, transforms

We will also need to access the shared folder. To do this, run drive.mount, click on the link, and enter the password.

**A.** Convolutional Layers and Maxpooling

Before defining the convolutional layers, let's examine two standard deep learning models (vgg16 and resnet50). In the final layers, you will recognize perceptrons ("classifier"), formed from the nn.Linear class. \\
But, in the first part of the network, you see the 'conv2d' layers that correspond to these famous convolutions.

In [ ]:
# Two deep neural networks: VGG16 and ResNet50

model = models.vgg16(pretrained=False)
# model = models.resnet50(pretrained=False)
print(model)

In the simplest case, in 1 dimension, the "convolution" of a neuron is defined by a relation of the form:
\begin{equation*}
output_i = bias + \sum_{j = 1}^n input_{i + j} \times kernel_{j}  \tag{1}
\end{equation*}
Here, $kernel$ represents a vector of size $n$ containing the parameters of the neuron. If, for example, $kernel$ is positive and has a sum of 1, it is a moving average. Finally, note that the classic convolution operator differs slightly ($input_{i - j}$ instead of $input_{i + j}$).

The PyTorch code is more complicated because:
- the input is generally a batch of images with multiple channels
- there is not just one neuron

The general form is defined [here](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d).

On this page, $C_{in}$, $C_{out}$, and $N$ correspond respectively to the numbers of input channels, layer neurons, and batch size.

The operator $\star$ also hides subtleties: you need to set how the kernel moves on the input (*stride*) and deal with border issues (*padding*). These aspects, which we will not dwell on, are easier to understand from the animations by [Vincent Dumoulin](https://github.com/vdumoulin/conv_arithmetic).

To illustrate the effect of a 2D convolutional layer with one neuron, let's load an RGB image.

**Upload the file 'cat.jpg' from the GitHub repository to Google Colab by dragging and dropping it into the Files Explorer on the left side.**

In [ ]:
! git clone https://github.com/relmonta/ml-student.git

In [ ]:
! ls ml-student/TP1

In [ ]:
from PIL import Image
root = 'ml-student/TP1'
path = os.path.join(root, 'cat.jpg')
image = Image.open(path).convert("RGB")
image = image.resize((256,256))
image

The "convolution" operators used by the `Conv2d` class are coded in the `torch.nn.functional` module. In the following lines, we define a Gaussian kernel that can then be applied to the image:

In [ ]:
import torch.nn.functional as F  # convolution layers

# Definition of a Gaussian kernel of size 10
x_range = torch.arange(-5, 5, 0.5)
y_range = torch.arange(-5, 5, 0.5)
xx, yy = torch.meshgrid(x_range, y_range)
var = 10

kernel =  (1./(2.*3.14*var)) * torch.exp( - (xx**2 + yy**2)/(2*var) )  # Gaussian

plt.imshow(kernel, cmap='gray')

In [ ]:
# Applying the same kernel to all channels:
kernel = kernel.repeat(1,3,1,1)                        # repeat for each R,G,B channel

# We added at the beginning the dimension associated with the image indexing in the batch ("batch dimension"):
print(kernel.shape)

In [ ]:
# Converting the 'cat.jpg' image to torch.tensor:
img = transforms.ToTensor()(image)
img = img.unsqueeze(dim=0)  # add the "batch dimension"

# Convolution:
output = F.conv2d(img, kernel)
fig = plt.figure()
plt.imshow(output[0,0,:,:], cmap='gray')

In practice, kernels of such large size are rarely used. For example, in a VGG, the kernels are of size 3*3. This is sufficient to extract interesting features, such as contours.

**Exercise 1:** Apply a [Prewitt filter](https://en.wikipedia.org/wiki/Prewitt_operator) to the image using PyTorch's `conv2d` (complete the following code).

In [ ]:
# kernel2 = ...
# kernel2 = ...
# YOUR CODE HERE
raise NotImplementedError()
#output1 = ..
#output2 = ..
# YOUR CODE HERE
raise NotImplementedError()
output = (output1**2 + output2**2).sqrt()

fig = plt.figure()
plt.imshow(output[0,0,:,:], cmap='gray')

However, the Prewitt filter cannot be encoded in a standard neural network: it involves a square root. The nonlinearities implemented in these networks are simpler:

- the ReLU function. It is simply the "positive part" function.
- maxpooling. It is a form of downsampling. In its most common form, the image is divided into 2*2 pixel squares, and the maximum value in each square is returned. The spatial dimensions of the output tensor are thus divided by two.  

In [ ]:
# Positive part: ReLU function
x = torch.rand(1,1,4,4) - 0.5
print(x)
print(x.relu())

# Maxpooling:
x = F.max_pool2d(x, kernel_size=2)
print(x)

**Exercise 2:** What is the size of the tensor at the output of layer 30 of vgg16, compared to that of the input image?
YOUR ANSWER HERE
**B.** Convolution Kernels After Training

Let's now observe the convolutional layers after training on a very large set of annotated images (~1M) from the ImageNet database ([http://www.image-net.org/challenges/LSVRC/2010/index](https://)). First, let's see what happens to the kernels associated with the 64 neurons in the first convolutional layer of a pre-trained ResNet50:

In [ ]:
# Before training:
model = models.resnet50(pretrained=False)
first_layer = model.conv1.weight.data

print(first_layer.shape)

plt.figure(figsize=(15, 15))
for i in range(first_layer.shape[0]):
    plt.subplot(8, 8, i+1) #
    plt.imshow(first_layer[i, 0, :, :], cmap='seismic')
    plt.axis('off')
plt.show()

In [ ]:
# After training:
model = models.resnet50(pretrained=True)
first_layer = model.conv1.weight.data

print(first_layer.shape)

plt.figure(figsize=(15, 15))
for i in range(first_layer.shape[0]):
    plt.subplot(8, 8, i+1) #
    plt.imshow(first_layer[i, 0, :, :], cmap='seismic')
    plt.axis('off')
plt.show()

Among these kernels, we find contour extractors similar to the Prewitt filter.  
Signal processing enthusiasts will even recognize patterns very close to the [Morlet wavelets](https://www.google.com/search?q=wavelet+morlet+2d&rlz=1C1AVFC_enFR826FR857&hl=fr&sxsrf=ALeKk01sWHdzUO6bRogEv0KFx2gRgOWz_Q:1610077971021&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiIq5fst4vuAhXPxYUKHaskDqAQ_AUoAXoECAUQAw&biw=915&bih=591).  
This is remarkable: through a simple gradient descent, effective filters for shape recognition have emerged.

**Exercise 3:** How many "neurons" does the first convolutional layer of a ResNet50 have (one kernel per neuron)?  
How many weights are there in a kernel?  
How many weights does this layer contain?

YOUR ANSWER HERE

In [ ]:
# This can be verified with the module.parameters() generator,
# applicable to any instance of torch.nn
nb_weights = 0
for parameter in model.conv1.parameters():
  # numel: counts the number of elements in a tensor
  nb_weights += torch.numel(parameter)

print(nb_weights)

**C.** Feature Maps

Now let's see what happens to the image as it passes through the network. Let's take VGG16, pass it on our cat image, and see how the signal changes as it propagates through the network. First, let's see if the network recognizes the correct class among the thousand classes in the dataset. The list of classes is [here](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

In [ ]:
image = Image.open(path).convert("RGB")
image = image.resize((256,256))

img = transforms.ToTensor()(image)

img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
img = img.unsqueeze(dim=0)

In [ ]:
model = models.vgg16(pretrained=True)

In [ ]:
# Raw output
output = model(img)

# Softmax function
output = output.softmax(dim=1).detach()

# Prediction
_, c  =torch.max(output, dim=1)
print(c)

# "Probabilities" associated with classes
plt.plot(output.squeeze())

You may notice that by running the last cell several times, the output probabilities differ even though the input does not change. This is due to "dropout" (check `print(model)`), which deactivates a random subset of the classifier neurons (this operation helps combat **overfitting**).  

To disable "dropout" and freeze the network, switch to "eval" mode with the command:  

*model.eval()*

Now, let's visualize the signal at the output of a convolutional layer. For this, we can use the [*hook*](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html) command. The channels of these intermediate signals are called **feature maps**. Let's see the feature maps associated with the first convolutional layer:

In [ ]:
z = []
# Function that stores in the list z the "feature maps"
def store_layer_output(model, input, output):
        z.append(output.detach())


model.features[0].register_forward_hook(store_layer_output)
model.features[10].register_forward_hook(store_layer_output)
model.features[17].register_forward_hook(store_layer_output)
model.features[28].register_forward_hook(store_layer_output)

output = model(img)

In [ ]:
for fm in z:
  print(fm.shape)

**Exercise 4:** Visualize the feature maps of layers 0, 10, 17, 28. You will notice that at level 10, the neuron's response is often specific to a particular trait.

In [ ]:
feature_maps = z[1]

print(feature_maps.shape)
plt.figure(figsize=(20, 17))
# for i in range(...):
#    plt.subplot(...)
#    plt.imshow(...)
#    plt.axis('off') # don't show plot axis
# YOUR CODE HERE
raise NotImplementedError()
plt.show()


Overlay on the original image:

In [ ]:
import cv2
# load image
# YOUR CODE HERE
raise NotImplementedError()

f, axes = plt.subplots(8,8)
for i in range(64):
    ax = axes[i//8][i%8]
    # fm1 = feature map
    # YOUR CODE HERE
    raise NotImplementedError()
    fm1 = cv2.resize(fm1, (image.size))
    fm1 /= 0.5*np.max(fm1)
    fm1 = np.uint8(fm1*255)
    fm1 = cv2.applyColorMap(fm1,  cv2.COLORMAP_HOT)
    fm1 = Image.fromarray(fm1)

    superp = Image.blend(image, fm1, 0.5)
    ax.axis('off')
    ax.imshow(superp)
f.set_figheight(20)
f.set_figwidth(20)